In [1]:
import json
import clip
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F

# CLIPSeg Experiments

In [2]:
from clipseg_model import CLIPSeg

with open('concepts/CUB/parts.txt') as fp:
    part_texts = ['bird ' + word for word in fp.read().splitlines()]

state_dict = torch.load('checkpoints/clipseg_pascub_ft.pt')
model = CLIPSeg(part_texts=part_texts, state_dict=state_dict)

Some weights of CLIPSegForImageSegmentation were not initialized from the model checkpoint at CIDAS/clipseg-rd64-refined and are newly initialized: ['non_object_embedding', 'text_adapter.fc.0.weight', 'text_adapter.fc.2.weight', 'tunable_linear.weight', 'visual_adapter.fc.0.weight', 'visual_adapter.fc.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'clipseg.configuration_clipseg.CLIPSegConfig'> 512


In [3]:
import os
from torch.utils.data import DataLoader
from data.cub_dataset_v2 import CUBDatasetSimple

def collate_fn(batch):
    image_list, label_list = list(zip(*batch))
    return image_list, torch.stack(label_list)

dataset_train = CUBDatasetSimple(os.path.join('datasets', 'CUB'), split='train')
dataloader_train = DataLoader(dataset=dataset_train, collate_fn=collate_fn, batch_size=2, shuffle=True)
dataloader_train_iter = iter(dataloader_train)

In [4]:
with torch.no_grad():
    image_list, targets = next(dataloader_train_iter)
    loss = model(image_list, targets)
    print(loss)

tensor(13.1073)


# Mahalanobis Distance Experiments

In [ ]:
clip_model, clip_preprocess = clip.load('RN50')

with open('concepts/CUB/concepts_processed.json', 'r') as fp:
    concept_dict = json.load(fp=fp)

unique_concepts = set()
for class_name, class_concepts in concept_dict.items():
    for concepts in class_concepts.values():
        unique_concepts.update(concepts)

unique_concepts = sorted(list(unique_concepts))

concepts_tokenized = clip.tokenize(unique_concepts)
with torch.no_grad():
    concepts_encoded = clip_model.encode_text(concepts_tokenized)

concepts_encoded_norm = F.normalize(concepts_encoded, dim=-1)
concepts_mean = torch.mean(concepts_encoded_norm, dim=0)
concepts_cov = torch.cov(concepts_encoded_norm.T)
sigma_inv = torch.inverse(concepts_cov)

sample = torch.randn(1024)
mahalanobis_dist = 1e-4 * (sample - concepts_mean) @ sigma_inv @ (sample - concepts_mean)

sample = concepts_encoded_norm[100]
mahalanobis_dist = 1e-4 * (sample - concepts_mean) @ sigma_inv @ (sample - concepts_mean)